## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter, defaultdict
# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison

# Stopwords
sw = stopwords.words("english")

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

def remove_stop(tokens) :
    # modify this function to remove stopwords
    tokens = [t for t in tokens if t not in sw and 'https' not in t]
    
    tokens = [t for t in tokens if t.isalpha() or t.isnumeric()]
    return tokens
 
def remove_punctuation(text, punct_set=punctuation) : 
    return("".join([ch for ch in text if ch==' ' or (ch not in punct_set and ch.isalnum())]))

def tokenize(text) : 
    
    # modify this function to return tokens
    token = text.split()
    return token

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return tokens
my_pipeline = [str.lower, remove_punctuation, tokenize,remove_stop]

In [2]:
convention_db = sqlite3.connect(r'C:\Users\ahzha\Desktop\2020_Conventions.db')
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [3]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                             SELECT text,party FROM conventions
                            ''')

for row in query_results :
    # store the results in convention_data
    
    text = row[0]
    
    text = ' '.join(prepare(text,my_pipeline))
    convention_data.append([text,row[1]])

Let's look at some random entries and see if they look right. 

In [4]:
random.choices(convention_data,k=10)

[['cant tell grateful opportunity speak today long ago life running police fearing police avoiding police anything police done personally due animosity allowed grow inside making believe enemy today praise god filled hope proud american citizen given second chance transformation began prison cell ive found threetime convicted felon facing yet another sentence gave life jesus made promise would spend rest days helping others like first help support came unlikeliest places fbi agent arrested richard beasley dear friend source encouragement throughout entire journey',
  'Republican'],
 ['incredible first lady', 'Republican'],
 ['humankind long viewed soul vital center core essence existence soul makes us us finest hours americas soul animated proposition created equal imperative ensure treated equally yet america mix light shadow seneca falls selma stonewall dwell american soul impulses given us slavery segregation systemic discrimination',
  'Democratic'],
 ['deported september come back

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [5]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2373 as features in the model.


In [6]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    ret_dict = dict()
    for token in text.split():
        if token in fw:
            ret_dict[token] = True
    return(ret_dict)

In [7]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [8]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [9]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [10]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.498


In [11]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### Observations
- Most of the important features belongs to Republican party.

- The model accuracy is ~50% only. China appears more in Rep vs. Dem.


## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [12]:
cong_db = sqlite3.connect(r'C:\Users\ahzha\Desktop\congressional_data.db')
cong_cur = cong_db.cursor()

In [13]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [14]:
from tqdm import tqdm
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.
for row in tqdm(results) :
    # store the results in convention_data
    
    text = row[2]
    
    text = ' '.join(prepare(text,my_pipeline))
    tweet_data.append([text,row[1]])

100%|████████████████████████████████████████████████████████████████████████| 664656/664656 [01:10<00:00, 9447.85it/s]


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [15]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [16]:
word_cutoff = 0

tokens = [w for t, p in tweet_data_sample for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 0, we have 119 as features in the model.


In [17]:
for tweet, party in tweet_data_sample :
    
    estimated_party = classifier.classify(conv_features(tweet,feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: bearlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: bgo tribe rallytogether
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: bapparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: make even greater kag
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: bwe cavs tie series 22 im repbarbaralee scared roadtovictory
Actual party is Democratic and our classifer says Republ

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [18]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.seed(20220507)
random.shuffle(tweet_data)


word_cutoff = 5

tokens = [w for t, p in tweet_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")


for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet,feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

With a word cutoff of 5, we have 47065 as features in the model.


In [19]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3678, 'Democratic': 714}),
             'Democratic': defaultdict(int,
                         {'Republican': 4620, 'Democratic': 990})})

In [20]:
sum(results['Republican'].values())

4392

In [21]:
sum(results['Democratic'].values())

5610

### Reflections

- For the Republican party, there are ~15% false predictions.
- For Democratic party, there are ~80% false predictions. The accuracy is much lower here.